# PRIMERA FUENTE DE DATOS: World Bank

In [ ]:
# - requests: para obtener datos de Internet.
# - pandas: para organizar los datos en tablas.
# - json: para trabajar con información en formato JSON.
import requests
import pandas as pd
import json

# URL de la API del Banco Mundial donde están los datos de anemia infantil.
base_url = "http://api.worldbank.org/v2/country/ALL/indicator/SH.ANM.CHLD.ZS"

# Indicamos a la API que queremos los datos en formato JSON.
params = {
    "format": "json"
}

# Creamos una lista vacía donde guardaremos los datos.
all_data = []

# Enviamos la solicitud a la API.
response = requests.get(base_url, params=params)

# Verificamos si la solicitud fue exitosa (código 200 significa éxito).
if response.status_code == 200:
    # Convertimos la respuesta de la API en un formato que Python pueda entender (JSON).
    data = response.json()
    
    # Revisamos si hay datos útiles en la respuesta.
    if len(data) > 1:  # Los datos que necesitamos están en la segunda parte de la respuesta.
        all_data = data[1]  # Guardamos esos datos en nuestra lista.
    else:
        print("No se encontraron datos en la respuesta.")
else:
    # Si ocurre un error, mostramos el código de error.
    print("Error al obtener los datos:", response.status_code)

# Ahora, organizamos los datos en una tabla usando pandas.
df_worldbank = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV (una hoja de cálculo).
output_file = "world_bank_anemia.csv"
df_worldbank.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos el archivo sin incluir índices.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que se creó el archivo.

# SEGUNDA FUENTE: Global Health Observatory

In [ ]:
import requests
import pandas as pd
import json

# URL de la API para datos de nutrición y anemia infantil.
base_url = "https://ghoapi.azureedge.net/api/NUTRITION_ANAEMIA_CHILDREN_NUM"

# Configuramos los parámetros para obtener los datos en partes (paginación):
# - "$top": cuántos registros obtener por solicitud.
# - "$skip": cuántos registros saltar para la siguiente solicitud.
params = {
    "$top": 1000,  # Pedimos 1000 registros por solicitud.
    "$skip": 0     # Empezamos desde el inicio.
}

# Lista vacía para guardar todos los datos.
all_data = []

# Usamos un ciclo para seguir pidiendo datos hasta que no queden más.
while True:
    # Hacemos una solicitud a la API con los parámetros actuales.
    response = requests.get(base_url, params=params)
    
    # Si la solicitud fue exitosa:
    if response.status_code == 200:
        # Convertimos la respuesta en un formato JSON y agregamos los datos a nuestra lista.
        data = response.json()
        all_data.extend(data["value"])  # Extendemos la lista con los nuevos datos.
        
        # Si la cantidad de datos obtenidos es menor que "$top", significa que no hay más datos.
        if len(data["value"]) < params["$top"]:
            break  # Terminamos el ciclo.
        
        # Si hay más datos, incrementamos "$skip" para pedir la siguiente página.
        params["$skip"] += params["$top"]
    else:
        # Si ocurre un error, mostramos el código de error y terminamos el ciclo.
        print("Error al obtener los datos:", response.status_code)
        break

# Organizamos los datos en una tabla con pandas.
df_anemia = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV.
output_file = "nutrition_anemia_children.csv"
df_anemia.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos el archivo.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que se creó el archivo.

# TERCERA FUENTE: DHS Program

### ENCUESTA 1: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON CUALQUIER TIPO DE ANEMIA

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS para obtener datos de nutrición (anemia en este caso).
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_ANY"

# Parámetros que se enviarán a la API:
# - "perpage": número máximo de registros por solicitud (aquí pedimos 1000).
# - "page": indica el número de la página que estamos solicitando (empezamos en la 1).
params = {
    "perpage": 1000,  # Máximo de registros por página.
    "page": 1         # Comenzamos desde la página 1.
}

# Creamos una lista vacía para almacenar todos los datos obtenidos de la API.
all_data = []

# Usamos un ciclo para descargar todos los datos disponibles en la API.
while True:
    # Realizamos una solicitud a la API con los parámetros actuales.
    response = requests.get(base_url, params=params)
    
    # Verificamos si la solicitud fue exitosa (código de respuesta 200).
    if response.status_code == 200:
        # Convertimos la respuesta de la API a formato JSON (fácil de manejar en Python).
        data = response.json()
        
        # Extraemos los datos dentro de la clave "Data" y los agregamos a nuestra lista.
        # Usamos "get" para evitar errores si la clave no existe.
        all_data.extend(data.get("Data", []))
        
        # Verificamos si hemos llegado al final de los datos:
        # Si la cantidad de datos obtenidos es menor que el límite "perpage",
        # significa que no hay más páginas que consultar.
        if len(data.get("Data", [])) < params["perpage"]:
            break  # Salimos del ciclo porque ya no hay más datos.
        
        # Si aún hay más datos, incrementamos el número de página para la siguiente solicitud.
        params["page"] += 1
    else:
        # Si ocurre un error (código distinto de 200), mostramos el código de error y terminamos.
        print("Error al obtener los datos:", response.status_code)
        break

# Una vez descargados todos los datos, los organizamos en una tabla con Pandas.
df_any = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV llamado "df_any_anemia.csv".
output_file = "df_any_anemia.csv"
df_any.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos sin incluir índices y en UTF-8.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que el archivo se creó.

### ENCUESTA 2: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA LEVE

In [ ]:
import requests
import pandas as pd
import json
# URL de la API del DHS para obtener datos de anemia infantil leve.
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_MLD"

# Parámetros que enviamos a la API:
# - "perpage": cantidad máxima de datos que queremos recibir por solicitud (1000 aquí).
# - "page": indica el número de página que solicitamos (iniciamos desde la página 1).
params = {
    "perpage": 1000,  # Pedimos hasta 1000 registros por página.
    "page": 1         # Empezamos desde la primera página.
}

# Creamos una lista vacía para guardar todos los datos descargados de la API.
all_data = []

# Usamos un ciclo para realizar solicitudes a la API hasta que descarguemos todos los datos.
while True:
    # Realizamos la solicitud a la API con los parámetros configurados.
    response = requests.get(base_url, params=params)
    
    # Verificamos si la solicitud fue exitosa (código 200 indica éxito).
    if response.status_code == 200:
        # Convertimos la respuesta en formato JSON (fácil de manejar en Python).
        data = response.json()
        
        # Agregamos los datos obtenidos (clave "Data") a nuestra lista.
        # Usamos "get" para evitar errores si la clave no existe.
        all_data.extend(data.get("Data", []))
        
        # Verificamos si ya no hay más datos disponibles:
        # Si el número de datos recibidos es menor al límite "perpage", hemos llegado al final.
        if len(data.get("Data", [])) < params["perpage"]:
            break  # Salimos del ciclo porque no hay más datos.
        
        # Si aún hay más datos, pasamos a la siguiente página incrementando el número de página.
        params["page"] += 1
    else:
        # Si ocurre un error (respuesta distinta de 200), mostramos un mensaje con el código de error.
        print("Error al obtener los datos:", response.status_code)
        break

# Cuando terminamos de descargar los datos, los organizamos en una tabla usando pandas.
df_mld = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV llamado "df_mld_anemia.csv".
output_file = "df_mld_anemia.csv"
df_mld.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos sin índices y en formato UTF-8.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que el archivo fue creado.

Archivo CSV creado exitosamente: df_mld_anemia.csv


### ENCUESTA 3: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA MODERADA

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS para obtener datos de anemia infantil moderada.
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_MOD"

# Parámetros que enviamos a la API:
# - "perpage": define el número máximo de registros a recibir por solicitud (en este caso, 1000).
# - "page": indica el número de página que se solicita (empezamos en la página 1).
params = {
    "perpage": 1000,  # Pedimos hasta 1000 registros por página.
    "page": 1         # Comenzamos desde la primera página.
}

# Lista vacía para almacenar todos los datos descargados de la API.
all_data = []

# Ciclo para realizar solicitudes repetidas hasta que se descarguen todos los datos disponibles.
while True:
    # Realizamos una solicitud GET a la API utilizando la URL base y los parámetros definidos.
    response = requests.get(base_url, params=params)
    
    # Verificamos si la solicitud fue exitosa (código de estado 200).
    if response.status_code == 200:
        # Convertimos la respuesta en formato JSON (fácil de manipular en Python).
        data = response.json()
        
        # Extraemos los datos de la clave "Data" y los agregamos a la lista "all_data".
        # Usamos "get" para evitar errores si la clave no está presente.
        all_data.extend(data.get("Data", []))
        
        # Verificamos si hemos llegado al final de los datos disponibles:
        # Si el número de registros recibidos es menor que "perpage", significa que no hay más páginas.
        if len(data.get("Data", [])) < params["perpage"]:
            break  # Salimos del ciclo porque ya no hay más datos.
        
        # Si todavía hay más datos, incrementamos el número de página para la siguiente solicitud.
        params["page"] += 1
    else:
        # Si ocurre un error (código diferente de 200), mostramos el código de error y detenemos el proceso.
        print("Error al obtener los datos:", response.status_code)
        break

# Organizamos los datos descargados en una tabla (DataFrame) usando pandas.
df_mod = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV llamado "df_mod_anemia.csv".
output_file = "df_mod_anemia.csv"
df_mod.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos sin incluir índices y en formato UTF-8.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que el archivo fue creado correctamente.

Archivo CSV creado exitosamente: df_mod_anemia.csv


### ENCUESTA 4: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA GRAVE

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS (Nutrición - Anemia infantil moderada)
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_SEV"

# Definir los parámetros para la solicitud (si soporta paginación)
params = {
    "perpage": 1000,  # Número de registros por página (se solicita un máximo de 1000 registros por solicitud)
    "page": 1         # Página inicial, comenzamos desde la primera página de resultados
}

# Lista para almacenar todos los datos que se vayan obteniendo
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos (paginación)
while True:
    # Realizar la solicitud a la API utilizando los parámetros definidos
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta en formato JSON para manejar los datos
        data = response.json()
        
        # Extraer la lista de datos ("Data") de la respuesta JSON y agregarla a all_data
        all_data.extend(data.get("Data", []))
        
        # Verificar si hemos recibido menos registros de los solicitados, lo que indica que hemos llegado al final
        if len(data.get("Data", [])) < params["perpage"]:
            break
        
        # Incrementar el número de página para la siguiente solicitud
        params["page"] += 1
    else:
        # En caso de error, imprimir el código de estado de la respuesta
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos obtenidos (all_data) a un DataFrame de Pandas para facilitar su análisis
df_sev = pd.json_normalize(all_data)

# Guardar el DataFrame en un archivo CSV
output_file = "df_sev_anemia.csv"
df_sev.to_csv(output_file, index=False, encoding="utf-8")

# Imprimir mensaje de éxito indicando que el archivo CSV fue creado correctamente
print(f"Archivo CSV creado exitosamente: {output_file}")


Archivo CSV creado exitosamente: df_sev_anemia.csv


# CUARTA FUENTE: KAGGLE

In [ ]:
pip install kaggle

In [ ]:
import json
import os

# Ruta al archivo 'kaggle.json' que contiene las credenciales de acceso a la API de Kaggle
kaggle_file_path = 'kaggle.json'

# Leer el archivo JSON que contiene las credenciales
with open(kaggle_file_path, 'r') as file:
    kaggle_api = json.load(file)

# Extraer el 'username' y la 'key' de las credenciales de Kaggle
kaggle_username = kaggle_api['username']  # Obtiene el nombre de usuario de Kaggle
kaggle_key = kaggle_api['key']  # Obtiene la clave de la API de Kaggle

# Configurar las variables de entorno con las credenciales para acceder a la API de Kaggle
os.environ['KAGGLE_USERNAME'] = kaggle_username  # Establece la variable de entorno para el nombre de usuario
os.environ['KAGGLE_KEY'] = kaggle_key  # Establece la variable de entorno para la clave de la API

# Imprimir un mensaje de confirmación indicando que las credenciales han sido configuradas correctamente
print("Credenciales de Kaggle configuradas correctamente.")

In [ ]:
import subprocess

# Ejecutar el comando para listar datasets relacionados con "child anemia" en Kaggle y capturar la salida
result = subprocess.run(
    ["kaggle", "datasets", "list", "-s", "child anemia"],  # El comando Kaggle para buscar datasets
    capture_output=True,  # Captura tanto la salida estándar como los errores
    text=True  # Decirle a subprocess que el resultado debe ser tratado como texto (en lugar de bytes)
)

# Imprimir la salida del comando ejecutado
print(result.stdout)  # Muestra el resultado del comando (en este caso, la lista de datasets)



### DATA 1: Factores que afectan el nivel de anemia en los niños (Estudio en Nigeria)


In [ ]:
import subprocess

# Nombre del dataset a descargar desde Kaggle
dataset_name = "adeolaadesina/factors-affecting-children-anemia-level"

# Ejecutar el comando para descargar y descomprimir el dataset
subprocess.run(["kaggle", "datasets", "download", "-d", dataset_name, "--unzip"])


In [ ]:
import pandas as pd

# Verifica el nombre del archivo descargado y ajústalo aquí
csv_file = "children anemia.csv"  # El nombre del archivo CSV descargado

# Leer el archivo CSV en un DataFrame de Pandas
df_nigeria = pd.read_csv(csv_file)

# Confirmar que los datos se han leído correctamente
print("Datos extraídos exitosamente de Kaggle (Caso Nigeria)")


### DATA 2: Encuesta Nacional de Familia y Salud (Estudio en ¿India?)

In [ ]:
import subprocess

# Nombre del dataset que quieres descargar desde Kaggle
dataset_name = "ravisinghiitbhu/nfhs5"  # Identificador del dataset

# Ejecutar el comando para descargar y descomprimir el dataset
subprocess.run(["kaggle", "datasets", "download", "-d", dataset_name, "--unzip"])


In [ ]:
import pandas as pd

## Verifica el nombre del archivo descargado y ajústalo aquí
csv_file = "Final.csv"  # Nombre del archivo CSV descargado desde Kaggle

# Leer el archivo CSV en un DataFrame de Pandas
df_india = pd.read_csv(csv_file)

# Confirmar que los datos se han leído correctamente
print("Datos extraídos exitosamente de Kaggle (Caso India)")
